In [1]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizerFast, BatchEncoding, TFBertForTokenClassification
import numpy as np
from sklearn.metrics import recall_score, classification_report
import os 
import re

In [14]:
tokenizer = BertTokenizerFast.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [10]:
val_df = pd.read_csv('datasets/n2c2_val_dataset.csv')
# notes_df = pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/NOTEEVENTS.csv')

In [13]:
val_df

,Unnamed: 0,chunked_tokens,chunked_labels,label_ids,filename
0,16,[ 101 1647 2236 131 19538 1477 118 55...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",333-01.xml
1,17,[ 101 116 120 8347 1358 191 2875 11...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",333-01.xml
2,18,[ 101 1406 120 19538 1477 14516 9238 21...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",333-01.xml
3,19,[ 101 120 188 1477 119 1103 1762 11...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",333-01.xml
4,20,[ 101 5674 14046 20504 119 1156 5257 11...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",333-01.xml
...,...,...,...,...,...
127,6278,[ 101 1647 2236 131 18615 1568 118 49...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",185-01.xml
128,6279,[ 101 9097 120 8710 1106 2584 9964 1...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",185-01.xml
129,6280,[ 101 2047 2052 117 1649 1119 1110 22...,['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '...,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",185-01.xml
130,6281,[ 101 117 176 1193 19364 3269 117 1...,['O' 'I-MEDICATION' 'I-MEDICATION' 'I-MEDICATI...,"[18, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...",185-01.xml


In [ ]:
# test_df = pd.read_csv('../data/CAD_test.csv')

In [ ]:
subset = notes_df.sample(8)

In [ ]:
# subset = subset[subset['TEXT'].apply(lambda x: 'hyperten' in x[:510])]

In [ ]:
subset

In [9]:
subset['encoded'] = subset['TEXT'].apply(lambda x: tokenizer.encode_plus(x, truncation=True, is_split_into_words=False, padding='max_length', max_length=512)  )

In [10]:
subset

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,encoded
1206849,1224852,30105,107625.0,2194-12-22,2194-12-22 03:20:00,NaN,Radiology,CTA ABD & PELVIS,NaN,NaN,[**2194-12-22**] 3:20 AM\n CTA ABD & PELVIS ...,"[input_ids, token_type_ids, attention_mask]"
1666017,1689935,800,185742.0,2184-09-16,2184-09-16 12:55:00,2184-09-16 13:05:00,Nursing/other,Report,16025.0,NaN,Case Management Note\nSpoke to mom today on ph...,"[input_ids, token_type_ids, attention_mask]"
1303904,1336231,5628,158360.0,2101-04-09,2101-04-09 07:00:00,2101-04-09 07:02:00,Nursing/other,Report,16356.0,NaN,Resp Care\n\n\nPt was weaned to [**6-15**] dur...,"[input_ids, token_type_ids, attention_mask]"
1091550,1103582,43126,132026.0,2124-09-05,2124-09-05 16:11:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,"[**Last Name (LF) **],[**First Name3 (LF) 2764...","[input_ids, token_type_ids, attention_mask]"
1063864,1066975,45566,114477.0,2194-03-25,2194-03-25 08:24:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2194-3-25**] 8:24 AM\n CHEST (PORTABLE AP) ...,"[input_ids, token_type_ids, attention_mask]"
1718157,1746165,4971,164702.0,2154-09-22,2154-09-22 07:08:00,2154-09-22 07:09:00,Nursing/other,Report,15938.0,NaN,"NNP Physical Exam\nPE: pink, AFOF, nasal CPAP ...","[input_ids, token_type_ids, attention_mask]"
1403506,1417307,11852,126240.0,2114-05-05,2114-05-05 05:48:00,2114-05-05 06:25:00,Nursing/other,Report,21570.0,NaN,NPN 1900-0730\nReview of Systems:\n\nResp: Pt...,"[input_ids, token_type_ids, attention_mask]"
697053,696955,43258,154238.0,2126-08-16,2126-08-16 20:46:00,2126-08-16 20:46:01,Nursing,Nursing Progress Note,16934.0,NaN,FULL CODE\n,"[input_ids, token_type_ids, attention_mask]"


In [11]:
val_inputs_dict = {'input_ids':np.array([x['input_ids'] for x in subset['encoded']])}

In [12]:
val_inputs_dict

{'input_ids': array([[  101,   164,   115, ...,   168,   168,   102],
        [  101,  1692,  2635, ...,     0,     0,     0],
        [  101,  1231, 20080, ...,     0,     0,     0],
        ...,
        [  101,   183,  1179, ...,     0,     0,     0],
        [  101,   183,  1643, ...,     0,     0,     0],
        [  101,  1554,  3463, ...,     0,     0,     0]])}

In [13]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    val_inputs_dict
))

In [14]:
model = TFBertForTokenClassification.from_pretrained('NER/models/clinicalbert_batchsize8_complete_brat_test2.h5')

All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at NER/models/clinicalbert_batchsize8_complete_brat_test2.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [15]:
outputs=model.predict(val_dataset.batch(8))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x2aaab3b58b38> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x2aaab3b58b38> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



In [16]:
predictions = tf.argmax(outputs[0], axis=2)

In [17]:
prediction_probabilities = tf.reduce_max(outputs[0], axis=2)

In [18]:
labels_list =['B-CAD', 'B-DIABETES', 'B-FAMILY_HIST', 'B-HYPERLIPIDEMIA', 'B-HYPERTENSION', 'B-MEDICATION', 'B-OBESE', 'B-PHI', 'B-SMOKER', 'I-CAD', 'I-DIABETES', 'I-FAMILY_HIST', 'I-HYPERLIPIDEMIA', 'I-HYPERTENSION', 'I-MEDICATION', 'I-OBESE', 'I-PHI', 'I-SMOKER', 'O']

In [19]:
reverse_labels_dict = {i:k for i,k in enumerate(labels_list)}

In [20]:
val_results_df = pd.DataFrame([list(predictions.numpy())]).transpose()

In [21]:
val_results_df['pred_probs'] = list(prediction_probabilities.numpy())

In [12]:
val_df['tokens'] = list(val_df['encoded'].apply(lambda x: [tokenizer.decode([y]) for y in x['input_ids']]) )

KeyError: 'encoded'

In [23]:
val_results_df.columns=['class_nums','pred_probs','tokens']

In [24]:
val_results_df['classes']=val_results_df['class_nums'].apply(lambda x: [reverse_labels_dict[y] for y in x] )

In [25]:
def unravel(lst):
    return [i for j in lst for i in j]

In [26]:
# i =0
# result = pd.DataFrame(list(zip(val_results_df.iloc[i]['tokens'], val_results_df.iloc[i]['classes'],val_results_df.iloc[i]['pred_probs'])), columns=['tokens', 'token_preds', 'pred_score']).head(50)
# result[result['token_preds'].apply(lambda x: x!='O')]

In [33]:
val_results_df['ROW_ID']= list(subset['ROW_ID'])
val_results_df['TEXT']= list(subset['TEXT'])
# val_results_df['label']= subset['label']


In [34]:
val_results_df

,class_nums,pred_probs,tokens,classes,ROW_ID,TEXT
0,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.379699, 9.137501, 9.669785, 9.660312, 7.258...","[[CLS], [, *, *, 219, ##4, -, 12, -, 22, *, *,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1224852,[**2194-12-22**] 3:20 AM\n CTA ABD & PELVIS ...
1,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.37688, 9.798229, 9.504141, 9.688389, 9.7705...","[[CLS], case, management, note, spoke, to, mom...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1689935,Case Management Note\nSpoke to mom today on ph...
2,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.6337185, 5.799882, 6.9745283, 7.9186935, 7....","[[CLS], re, ##sp, care, p, ##t, was, we, ##ane...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1336231,Resp Care\n\n\nPt was weaned to [**6-15**] dur...
3,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[8.796605, 9.486436, 9.8582115, 9.755816, 9.69...","[[CLS], [, *, *, last, name, (, l, ##f, ), *, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1103582,"[**Last Name (LF) **],[**First Name3 (LF) 2764..."
4,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.526486, 9.191794, 9.796338, 9.770987, 7.587...","[[CLS], [, *, *, 219, ##4, -, 3, -, 25, *, *, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1066975,[**2194-3-25**] 8:24 AM\n CHEST (PORTABLE AP) ...
5,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.097584, 6.75105, 5.806907, 6.965637, 8.9083...","[[CLS], n, ##n, ##p, physical, exam, p, ##e, :...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1746165,"NNP Physical Exam\nPE: pink, AFOF, nasal CPAP ..."
6,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.406612, 5.5737896, 5.3524084, 5.8258305, 7....","[[CLS], n, ##p, ##n, 1900, -, 07, ##30, review...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1417307,NPN 1900-0730\nReview of Systems:\n\nResp: Pt...
7,"[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[9.841223, 8.560209, 8.505009, 8.667839, 9.231...","[[CLS], full, code, [SEP], [PAD], [PAD], [PAD]...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",696955,FULL CODE\n


In [ ]:
val_results_df.to_csv('results/test_set_ner_results.csv')

In [84]:
notes_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [2]:
import re

In [3]:
def getNumbers(str): 
    array = re.findall(r'[0-9]+.[0-9]+', str) 
    return [float(x) for x in array]

In [4]:
results=[]
for fn in os.listdir('results'):
    if 'cat_mimic' in fn:
        results.append(pd.read_csv('results/'+fn))
        break
mimic_results = pd.concat(results)        
# mimic_results = pd.read_csv('results/10000_random_mimic_ner_results.csv')
mimic_results['pred_probs']=mimic_results['pred_probs'].apply(lambda x: getNumbers(x))
mimic_results['tokens']=mimic_results['tokens'].apply(lambda x:eval(x))
mimic_results['classes']=mimic_results['classes'].apply(lambda x:eval(x))

In [5]:
# getNumbers(mimic_results['pred_probs'].iloc[0].strip())

In [6]:
mimic_results

,Unnamed: 0,class_nums,pred_probs,tokens,classes,ROW_ID,TEXT
0,0,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.71122, 9.497805, 9.582635, 9.59256, 9.62387...","[[CLS], 11, ##am, -, 2, pm, n, ##s, ##g, trans...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1567667,11AM-2 pm Nsg Transfer Note\npt is a 47 y.o ma...
1,1,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.876578, 9.063311, 9.084541, 9.589953, 9.431...","[[CLS], mi, ##cu, nursing, note, 7, ##a, -, 7,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1419760,MICU NURSING NOTE 7A-7PM\n\nNEURO: STATUS UNC...
2,2,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.020595, 9.292485, 9.050617, 9.034067, 9.808...","[[CLS], social, work, note, :, this, s, ##w, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1530417,SOCIAL WORK NOTE:\nThis SW was able to interac...
3,3,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.718318, 8.678089, 9.611098, 9.276952, 9.236...","[[CLS], respiratory, care, p, ##t, remains, in...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1663387,Respiratory Care\nPt remains intuabted (#8.0 E...
4,4,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.208955, 8.727775, 9.52534, 9.729515, 9.4908...","[[CLS], shift, events, :, -, br, ##on, ##cho, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",655703,Shift Events:\n -Bronchoscopy (BAL sent from...
...,...,...,...,...,...,...,...
9995,9995,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.443636, 5.8734837, 5.6606464, 6.182343, 6.6...","[[CLS], n, ##p, ##n, 7, ##a, -, 7, ##p, #, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1732598,NPN 7a-7p\n\n\n#1: [**Known lastname 2257**] r...
9996,9996,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.006546, 8.390697, 9.2759, 9.100227, 9.41769...","[[CLS], respiratory, therapy, t, ##rac, ##hed,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1470617,Respiratory Therapy\nTrached with a 8 Portex. ...
9997,9997,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.177918, 9.683085, 9.910935, 9.894538, 9.910...","[[CLS], t, ##si, ##cu, process, note, ne, ##ur...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1403657,"tsicu process note\nneuro: alert, intubated, f..."
9998,9998,[18 18 18 18 18 18 18 18 18 18 18 0 18 18 18 ...,"[9.210128, 5.741715, 6.6658573, 7.935575, 7.27...","[[CLS], re, ##sp, care, p, ##t, received, from...","[O, O, O, O, O, O, O, O, O, O, O, B-CAD, O, O,...",1463297,RESP CARE\nPT RECEIVED FROM OR FOR A CABG X 3....


In [7]:
nonempty = mimic_results[mimic_results['classes'].apply(lambda x:any([y is not 'O' for y in x]))]

In [8]:
nonempty['nono']= nonempty['classes'].apply(lambda x:[y  for y in x if y is not 'O'])

/gpfs/gsfs12/users/moldwinar/chiqa-fsl-ks/tr3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
nonempty

,Unnamed: 0,class_nums,pred_probs,tokens,classes,ROW_ID,TEXT,nono
0,0,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.71122, 9.497805, 9.582635, 9.59256, 9.62387...","[[CLS], 11, ##am, -, 2, pm, n, ##s, ##g, trans...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1567667,11AM-2 pm Nsg Transfer Note\npt is a 47 y.o ma...,"[B-DIABETES, I-DIABETES, B-HYPERTENSION, I-HYP..."
1,1,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.876578, 9.063311, 9.084541, 9.589953, 9.431...","[[CLS], mi, ##cu, nursing, note, 7, ##a, -, 7,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1419760,MICU NURSING NOTE 7A-7PM\n\nNEURO: STATUS UNC...,"[B-HYPERTENSION, I-HYPERTENSION, I-HYPERTENSIO..."
7,7,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.976654, 9.330947, 9.644093, 9.489644, 9.520...","[[CLS], de, ##cu, ##bit, ##us, ul, ##cer, (, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",716411,Decubitus ulcer (Present At Admission)\n Ass...,"[B-DIABETES, I-DIABETES, I-DIABETES, I-DIABETE..."
8,8,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.27766, 9.494106, 9.505979, 9.280949, 9.6398...","[[CLS], de, ##cu, ##bit, ##us, ul, ##cer, (, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",716382,Decubitus ulcer (Present At Admission)\n Ass...,"[B-DIABETES, I-DIABETES, I-DIABETES, I-DIABETE..."
9,9,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.741974, 9.561644, 9.588206, 9.343685, 9.596...","[[CLS], de, ##cu, ##bit, ##us, ul, ##cer, (, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",716408,Decubitus ulcer (Present At Admission)\n Ass...,"[B-DIABETES, I-DIABETES, I-DIABETES, I-DIABETE..."
...,...,...,...,...,...,...,...,...
9987,9987,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.889771, 6.7364707, 8.209423, 8.60113, 9.473...","[[CLS], re, ##sp, care, p, ##t, initially, on,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1397113,resp care\nPt initially on psv12/peep5 and 50%...,"[B-MEDICATION, I-MEDICATION]"
9988,9988,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.027238, 9.74931, 9.228898, 9.292762, 9.8673...","[[CLS], trauma, sic, ##u, nursing, note, curre...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1403656,TRAUMA SICU NURSING NOTE\n\nCURRENT REVIEW OF ...,"[I-MEDICATION, B-MEDICATION, I-MEDICATION]"
9990,9990,[18 18 18 18 18 18 18 18 18 18 18 18 18 1 10 ...,"[9.290181, 9.476991, 9.905147, 9.825234, 9.864...","[[CLS], t, ##si, ##cu, hp, ##i, :, hp, ##i, :,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-DIAB...",640061,"TSICU\n HPI:\n HPI: 52M with DM, CRF, CHF ...","[B-DIABETES, I-DIABETES, B-MEDICATION, I-MEDIC..."
9993,9993,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.231602, 7.6890035, 6.438387, 7.40239, 8.287...","[[CLS], cc, ##u, /, mi, ##cu, nursing, progres...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1470616,ccu/micu nursing progress note\ns/o: pls see c...,"[B-MEDICATION, I-MEDICATION, I-MEDICATION, B-H..."


In [10]:
for RF in ['HYPERTENSION', 'DIABETES','CAD', 'SMOKER', 'HYPERLIPIDEMIA', 'OBESE', 'MEDICATION','FAMILY_HIST']:
#     rf_df = nonempty[nonempty['nono'].apply(lambda x: set(x)==set(['B-'+RF,'I-'+RF]) )].sample(frac=1)
    rf_df = nonempty[nonempty['nono'].apply(lambda x: ('B-'+RF) in x) ].sample(frac=1)
    print(RF,len(rf_df))
#     rf_df[['TEXT']].to_csv('../concept_listfiles/cat_mimic'+RF)

HYPERTENSION 1712
DIABETES 481
CAD 252
SMOKER 107
HYPERLIPIDEMIA 126
OBESE 183
MEDICATION 2373
FAMILY_HIST 0


In [10]:
for RF in ['HYPERTENSION', 'DIABETES','CAD', 'SMOKER', 'HYPERLIPIDEMIA', 'OBESE', 'MEDICATION']:
    rf_df = nonempty[nonempty['nono'].apply(lambda x: set(x)==set(['B-'+RF,'I-'+RF]) )].sample(frac=1)
#     rf_df = nonempty[nonempty['nono'].apply(lambda x: ('B-'+RF) in x) ].sample(frac=1)
    print(RF,len(rf_df))
    rf_df[['TEXT']].to_csv('../concept_listfiles/cat_mimic_exclusive'+RF)

HYPERTENSION 46670
DIABETES 3334
CAD 5404
SMOKER 1146
HYPERLIPIDEMIA 698
OBESE 2187
MEDICATION 56736


In [8]:
empty = mimic_results[mimic_results['classes'].apply(lambda x:all([y is 'O' for y in x]))]

In [17]:
cats = notes_df[notes_df['CATEGORY'].apply(lambda x: x in ['Nursing/other','Nursing','General','Physician '])].dropna(subset=['CHARTDATE']).sample(frac=1)

In [18]:
cats['day'] = pd.to_datetime(cats['CHARTDATE']).dt.day_name()

In [20]:
for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    day_df = cats[cats['day']==day]
    day_df[['TEXT']].sample(2000).to_csv('../concept_listfiles/cat_mimic_day_'+day)

In [9]:
empty

,Unnamed: 0,class_nums,pred_probs,tokens,classes,ROW_ID,TEXT
2,2,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.020595, 9.292485, 9.050617, 9.034067, 9.808...","[[CLS], social, work, note, :, this, s, ##w, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1530417,SOCIAL WORK NOTE:\nThis SW was able to interac...
3,3,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.718318, 8.678089, 9.611098, 9.276952, 9.236...","[[CLS], respiratory, care, p, ##t, remains, in...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1663387,Respiratory Care\nPt remains intuabted (#8.0 E...
4,4,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.208955, 8.727775, 9.52534, 9.729515, 9.4908...","[[CLS], shift, events, :, -, br, ##on, ##cho, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",655703,Shift Events:\n -Bronchoscopy (BAL sent from...
5,5,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.192366, 8.826648, 9.572335, 9.782504, 9.556...","[[CLS], shift, events, :, -, br, ##on, ##cho, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",655699,Shift Events:\n -Bronchoscopy (BAL sent from...
6,6,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.020892, 9.549095, 9.429581, 9.763602, 9.256...","[[CLS], shift, note, :, 07, ##00, -, 1900, thi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1663388,Shift Note: 0700 - 1900\nThis is a gentleman w...
...,...,...,...,...,...,...,...
9994,9994,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.037367, 7.9961104, 8.402216, 7.8906846, 8.0...","[[CLS], [, *, *, name, (, ni, ), 65, ##2, *, *...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1407999,[**Name (NI) 652**]\nPT MAINTAINED ON A/C VENT...
9995,9995,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.443636, 5.8734837, 5.6606464, 6.182343, 6.6...","[[CLS], n, ##p, ##n, 7, ##a, -, 7, ##p, #, 1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1732598,NPN 7a-7p\n\n\n#1: [**Known lastname 2257**] r...
9996,9996,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.006546, 8.390697, 9.2759, 9.100227, 9.41769...","[[CLS], respiratory, therapy, t, ##rac, ##hed,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1470617,Respiratory Therapy\nTrached with a 8 Portex. ...
9997,9997,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.177918, 9.683085, 9.910935, 9.894538, 9.910...","[[CLS], t, ##si, ##cu, process, note, ne, ##ur...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1403657,"tsicu process note\nneuro: alert, intubated, f..."


In [26]:
1195978/10000

119.5978

In [12]:
for i in range(100):
    empty_sample = empty.sample(500)
    empty_sample[['TEXT']].to_csv('../concept_listfiles/empty_'+str(i))

,Unnamed: 0,class_nums,pred_probs,tokens,classes,ROW_ID,TEXT
3,3,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.350771, 8.516055, 9.401684, 9.366745, 9.716...","[[CLS], admission, date, :, [, *, *, 212, ##5,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41205,Admission Date: [**2125-1-22**] ...
14,14,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.413321, 8.409092, 9.397543, 9.339934, 9.673...","[[CLS], admission, date, :, [, *, *, 212, ##4,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41216,Admission Date: [**2124-3-14**] ...
17,17,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[8.836275, 7.789558, 8.873829, 8.312129, 8.704...","[[CLS], admission, date, :, [, *, *, 212, ##5,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",52891,Admission Date: [**2125-11-10**] Discha...
18,18,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.220997, 8.518022, 9.311862, 8.976128, 9.604...","[[CLS], admission, date, :, [, *, *, 217, ##7,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",52892,Admission Date: [**2177-5-20**] ...
22,22,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.335224, 8.5170555, 9.402229, 9.248004, 9.60...","[[CLS], admission, date, :, [, *, *, 215, ##7,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",52896,Admission Date: [**2157-12-31**] ...
...,...,...,...,...,...,...,...
9988,9988,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.216661, 8.631879, 9.47998, 9.31728, 9.68543...","[[CLS], admission, date, :, [, *, *, 217, ##6,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41190,Admission Date: [**2176-12-12**] ...
9989,9989,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.106506, 8.333713, 9.266536, 9.004337, 9.549...","[[CLS], admission, date, :, [, *, *, 215, ##6,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41191,Admission Date: [**2156-2-19**] ...
9992,9992,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.576324, 8.366075, 9.332402, 8.890347, 9.355...","[[CLS], admission, date, :, [, *, *, 215, ##1,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41194,Admission Date: [**2151-1-7**] D...
9995,9995,[18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 ...,"[9.019789, 8.415865, 9.248111, 8.919672, 9.450...","[[CLS], admission, date, :, [, *, *, 211, ##1,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",41197,Admission Date: [**2111-12-11**] ...


In [33]:
# family_hist =  nonempty[nonempty['nono'].apply(lambda x: 'B-FAMILY_HIST' in x and  ally not in x for y in )].sample(frac=1)

In [37]:
medication.iloc[46]['TEXT']

'Name:  [**Known lastname 15128**],[**Known firstname **] M                   Unit No:   [**Numeric Identifier 15129**]\n\nAdmission Date:  [**2188-3-27**]              Discharge Date:   [**2188-4-1**]\n\nDate of Birth:  [**2127-8-18**]             Sex:   M\n\nService: CARDIOTHORACIC\n\nAllergies:\nNo Known Allergies / Adverse Drug Reactions\n\nAttending:[**First Name3 (LF) 1543**]\nAddendum:\nMedication changes:\nPercocet changed to Oxycodone as patient c/o "sweats" with\nPercocet.\nAlbuterol inhaler added.\n\n\nDischarge Medications:\n1. rosuvastatin 20 mg Tablet Sig: One (1) Tablet PO DAILY\n(Daily).\nDisp:*30 Tablet(s)* Refills:*0*\n2. aspirin 81 mg Tablet, Delayed Release (E.C.) Sig: One (1)\nTablet, Delayed Release (E.C.) PO DAILY (Daily).\n3. metoprolol tartrate 25 mg Tablet Sig: Three (3) Tablet PO BID\n(2 times a day).\nDisp:*180 Tablet(s)* Refills:*2*\n4. lorazepam 0.5 mg Tablet Sig: One (1) Tablet PO Q4H (every 4\nhours) as needed for anxiety.\nDisp:*10 Tablet(s)* Refills:*0

In [174]:
i =12
result = pd.DataFrame(list(zip(nonempty.iloc[i]['tokens'], nonempty.iloc[i]['classes'],nonempty.iloc[i]['pred_probs'])), columns=['tokens', 'token_preds', 'pred_score'])
result[result['token_preds'].apply(lambda x: x is not 'O')]

,tokens,token_preds,pred_score
20,173,I-OBESE,2.7768552
62,r,B-MEDICATION,5.600804
63,##iss,I-MEDICATION,5.843225


In [175]:
nonempty.iloc[i]['TEXT']

nan

In [5]:
s=10000
split_notes = [notes_df.iloc[s*i:s*i+s] for i in range(int(len(notes_df)))]

In [6]:
sum([len(x) for x in split_notes])

2083180

In [7]:
len(notes_df)

2083180

In [4]:
notes_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [5]:
cats

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
308691,316234,29075,179159.0,2116-02-07,2116-02-07 14:08:00,2116-02-07 14:08:20,Nursing,Nursing Transfer Note,20808.0,NaN,"67M w/ h/o multiplemyeloma Dx [**2111**], neur..."
308692,316235,18082,181163.0,2156-03-12,2156-03-12 14:23:00,2156-03-12 14:23:59,Nursing,Nursing Transfer Note,15065.0,NaN,"[**Age over 90 52**]F with COPD on home O2, CA..."
308693,316236,18082,181163.0,2156-03-12,2156-03-12 14:28:00,2156-03-12 14:28:37,Nursing,Nursing Progress Note,15065.0,NaN,"[**Age over 90 52**]F with COPD on home O2, CA..."
308694,316237,16605,109285.0,2138-03-21,2138-03-21 15:02:00,2138-03-21 15:03:05,Physician,Physician Resident Progress Note,21203.0,NaN,Chief Complaint:\n 24 Hour Events:\n Continu...
308695,316238,29075,179159.0,2116-02-07,2116-02-07 15:37:00,2116-02-07 15:37:10,Physician,Physician Resident Progress Note,21203.0,NaN,Chief Complaint:\n 24 Hour Events:\n EGD d...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [2]:
import pandas as pd

In [3]:
for RF in ['HYPERTENSION', 'DIABETES','CAD', 'SMOKER', 'HYPERLIPIDEMIA', 'OBESE', 'MEDICATION']:
    print(RF)
    print(len(pd.read_csv('../concept_listfiles/cat_mimic_exclusive'+RF) ) )

HYPERTENSION
46670
DIABETES
3334
CAD
5404
SMOKER
1146
HYPERLIPIDEMIA
698
OBESE
2187
MEDICATION
56736


In [26]:
alldata = pd.concat([pd.read_pickle('datasets/n2c2_val_dataset.pk'), pd.read_pickle('datasets/n2c2_train_dataset.pk')])

In [27]:
def unravel(lst):
    return [i for j in lst for i in j]

In [29]:
alldata['labels'] = alldata['labels'].apply(lambda x: list(set(x)))

In [32]:
alldata = alldata.drop_duplicates(subset=['filename'], keep='first')

In [9]:
for label in 
alldata.groupby('filename')['chunked_labels'].apply(lambda x: unravel(list(x)))

filename
100-01.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
100-02.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
100-03.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
100-04.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
100-05.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
                                    ...                        
400-01.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
400-02.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
400-03.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
400-04.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
400-05.xml    [[, ', O, ',  , ', O, ',  , ', O, ',  , ', O, ...
Name: chunked_labels, Length: 1304, dtype: object

In [36]:
for label in set(unravel(list(alldata['labels']))):
    print(label)
    print(len( alldata[alldata['labels'].apply(lambda x: label in x)] ) )

MEDICATION
1148
HYPERTENSION
1075
before
1294
CAD
522
SMOKER
392
OBESE
241
DIABETES
903
HYPERLIPIDEMIA
593
FAMILY_HIST
39


In [40]:
alldata['filename'].sort_values()

5425    100-01.xml
5447    100-02.xml
5479    100-03.xml
5498    100-04.xml
5503    100-05.xml
           ...    
142     400-01.xml
143     400-02.xml
480     400-03.xml
527     400-04.xml
532     400-05.xml
Name: filename, Length: 1304, dtype: object